<a id='home'></a>
### purpose

gather performance scores from geno runs into one object

### notes

not all individual training jobs have finished. We will not run them to completion.

### outline

1. [get performance directories](#dir)

1. [combine performane information](#ret)

1. [save data as one object](#save)

In [1]:
from pythonimports import *
from myfigs import draw_xy, adjust_box_widths

import MVP_summary_functions as mvp
import runtime_API as rt

import seaborn as sns
import itertools as it

outerdir = '/work/lotterhos/brandon/ind_runtimes'

lview, dview = get_client(cluster_id='1708958269-nk3j', profile='lotterhos')

t0 = dt.now()  # notebook timer

rt.latest_commit()
session_info.show()

36 36
#########################################################
Today:	February 26, 2024 - 10:58:04 EST
python version: 3.8.5
conda env: mvp_env

Current commit of pythonimports:
commit 977f4ecb4bf51986ad24179b2609d4c63c5590a0  
Author: Brandon Lind <lind.brandon.m@gmail.com>  
Date:   Fri Feb 16 12:13:22 2024 -0500

Current commit of MVP_offsets:
commit 8b790072e7a46d7f58a30c40cf4660986a612764  
Author: Brandon Lind <lind.brandon.m@gmail.com>  
Date:   Fri Feb 2 13:55:49 2024 -0500

Current commit of MVP_runtime:
commit b24085be3441e41299da35731bc65e1b3eb25d81  
Author: Brandon Lind <lind.brandon.m@gmail.com>  
Date:   Tue Feb 20 09:43:50 2024 -0500
#########################################################



<a id='dir'></a>
# get performance directories

[top](#home)

In [2]:
ind_dirs = rt.dirs['ind']

ind_dirs

{'run_20220919_0-225': ['/work/lotterhos/brandon/ind_runtimes/run_20220919_0-225/00500',
  '/work/lotterhos/brandon/ind_runtimes/run_20220919_0-225/05000',
  '/work/lotterhos/brandon/ind_runtimes/run_20220919_0-225/10000',
  '/work/lotterhos/brandon/ind_runtimes/run_20220919_0-225/20000'],
 'run_20220919_225-450': ['/work/lotterhos/brandon/ind_runtimes/run_20220919_225-450/00500',
  '/work/lotterhos/brandon/ind_runtimes/run_20220919_225-450/05000',
  '/work/lotterhos/brandon/ind_runtimes/run_20220919_225-450/10000',
  '/work/lotterhos/brandon/ind_runtimes/run_20220919_225-450/20000'],
 'run_20220919_450-675': ['/work/lotterhos/brandon/ind_runtimes/run_20220919_450-675/00500',
  '/work/lotterhos/brandon/ind_runtimes/run_20220919_450-675/05000',
  '/work/lotterhos/brandon/ind_runtimes/run_20220919_450-675/10000',
  '/work/lotterhos/brandon/ind_runtimes/run_20220919_450-675/20000']}

In [3]:
res_dirs = defaultdict(list)
for rep, repdirs in ind_dirs.items():
    for d in repdirs:
        res_dirs[rep].append(f'{d}/gradient_forests/validation/corrs')
        
res_dirs

defaultdict(list,
            {'run_20220919_0-225': ['/work/lotterhos/brandon/ind_runtimes/run_20220919_0-225/00500/gradient_forests/validation/corrs',
              '/work/lotterhos/brandon/ind_runtimes/run_20220919_0-225/05000/gradient_forests/validation/corrs',
              '/work/lotterhos/brandon/ind_runtimes/run_20220919_0-225/10000/gradient_forests/validation/corrs',
              '/work/lotterhos/brandon/ind_runtimes/run_20220919_0-225/20000/gradient_forests/validation/corrs'],
             'run_20220919_225-450': ['/work/lotterhos/brandon/ind_runtimes/run_20220919_225-450/00500/gradient_forests/validation/corrs',
              '/work/lotterhos/brandon/ind_runtimes/run_20220919_225-450/05000/gradient_forests/validation/corrs',
              '/work/lotterhos/brandon/ind_runtimes/run_20220919_225-450/10000/gradient_forests/validation/corrs',
              '/work/lotterhos/brandon/ind_runtimes/run_20220919_225-450/20000/gradient_forests/validation/corrs'],
             'run_2022

In [4]:
res_files = []
for rep, resdirs in res_dirs.items():
    for d in resdirs:
        if not op.exists(d):
            continue
            
        num_loci = d.split('/')[6]
            
        files = fs(d, endswith='.pkl')
        res_files.extend(files)
        
        print(rep, num_loci, len(files))

run_20220919_0-225 00500 225
run_20220919_0-225 05000 225
run_20220919_0-225 10000 225
run_20220919_0-225 20000 121
run_20220919_225-450 00500 225
run_20220919_225-450 05000 225
run_20220919_225-450 10000 225
run_20220919_225-450 20000 113
run_20220919_450-675 00500 225
run_20220919_450-675 05000 225
run_20220919_450-675 10000 225
run_20220919_450-675 20000 109


<a id='ret'></a>
# combine performance information

[top](#home)

In [5]:
def process_performance(pkl, source='ind'):
    import runtime_API as rt
    
    rt.params = params
    
    return rt.process_performance_pkl(pkl, source=source)

params = mvp.read_params_file()
dview['params'] = params

100%|███████████████| 2250/2250 [00:02<00:00, 817.95it/s]


In [6]:
for pkl in res_files[:1]:
    x = process_performance(pkl, source='ind')
    
x

,garden,score,final_LA,glevel,plevel,pleio,slevel,landscape,popsize,migration,simulation_level,rep,num_loci,seed,demography
0,1,-0.844195,0.500854,highly-polygenic,1-trait,no pleiotropy,NaN,Est-Clines,N-cline-center-to-edge,m-constant,highly-polygenic_1-trait_no pleiotropy_nan_Est...,0-225,00500,1231094,N-cline-center-to-edge_m-constant
1,2,-0.845280,0.500854,highly-polygenic,1-trait,no pleiotropy,NaN,Est-Clines,N-cline-center-to-edge,m-constant,highly-polygenic_1-trait_no pleiotropy_nan_Est...,0-225,00500,1231094,N-cline-center-to-edge_m-constant
2,3,-0.845522,0.500854,highly-polygenic,1-trait,no pleiotropy,NaN,Est-Clines,N-cline-center-to-edge,m-constant,highly-polygenic_1-trait_no pleiotropy_nan_Est...,0-225,00500,1231094,N-cline-center-to-edge_m-constant
3,4,-0.846994,0.500854,highly-polygenic,1-trait,no pleiotropy,NaN,Est-Clines,N-cline-center-to-edge,m-constant,highly-polygenic_1-trait_no pleiotropy_nan_Est...,0-225,00500,1231094,N-cline-center-to-edge_m-constant
4,5,-0.847179,0.500854,highly-polygenic,1-trait,no pleiotropy,NaN,Est-Clines,N-cline-center-to-edge,m-constant,highly-polygenic_1-trait_no pleiotropy_nan_Est...,0-225,00500,1231094,N-cline-center-to-edge_m-constant
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,-0.841013,0.500854,highly-polygenic,1-trait,no pleiotropy,NaN,Est-Clines,N-cline-center-to-edge,m-constant,highly-polygenic_1-trait_no pleiotropy_nan_Est...,0-225,00500,1231094,N-cline-center-to-edge_m-constant
96,97,-0.841254,0.500854,highly-polygenic,1-trait,no pleiotropy,NaN,Est-Clines,N-cline-center-to-edge,m-constant,highly-polygenic_1-trait_no pleiotropy_nan_Est...,0-225,00500,1231094,N-cline-center-to-edge_m-constant
97,98,-0.842356,0.500854,highly-polygenic,1-trait,no pleiotropy,NaN,Est-Clines,N-cline-center-to-edge,m-constant,highly-polygenic_1-trait_no pleiotropy_nan_Est...,0-225,00500,1231094,N-cline-center-to-edge_m-constant
98,99,-0.844158,0.500854,highly-polygenic,1-trait,no pleiotropy,NaN,Est-Clines,N-cline-center-to-edge,m-constant,highly-polygenic_1-trait_no pleiotropy_nan_Est...,0-225,00500,1231094,N-cline-center-to-edge_m-constant


In [7]:
# process in parallel

jobs = []
for pkl in res_files:
    jobs.append(
        lview.apply_async(
            process_performance, pkl, **{'source' : 'ind'}
        )
    )
    
watch_async(jobs)

all_data = pd.concat([j.r for j in jobs])

all_data


Watching 2368 jobs ...


100%|███████████████| 2368/2368 [00:56<00:00, 42.10it/s] 


,garden,score,final_LA,glevel,plevel,pleio,slevel,landscape,popsize,migration,simulation_level,rep,num_loci,seed,demography
0,1,-0.844195,0.500854,highly-polygenic,1-trait,no pleiotropy,NaN,Est-Clines,N-cline-center-to-edge,m-constant,highly-polygenic_1-trait_no pleiotropy_nan_Est...,0-225,00500,1231094,N-cline-center-to-edge_m-constant
1,2,-0.845280,0.500854,highly-polygenic,1-trait,no pleiotropy,NaN,Est-Clines,N-cline-center-to-edge,m-constant,highly-polygenic_1-trait_no pleiotropy_nan_Est...,0-225,00500,1231094,N-cline-center-to-edge_m-constant
2,3,-0.845522,0.500854,highly-polygenic,1-trait,no pleiotropy,NaN,Est-Clines,N-cline-center-to-edge,m-constant,highly-polygenic_1-trait_no pleiotropy_nan_Est...,0-225,00500,1231094,N-cline-center-to-edge_m-constant
3,4,-0.846994,0.500854,highly-polygenic,1-trait,no pleiotropy,NaN,Est-Clines,N-cline-center-to-edge,m-constant,highly-polygenic_1-trait_no pleiotropy_nan_Est...,0-225,00500,1231094,N-cline-center-to-edge_m-constant
4,5,-0.847179,0.500854,highly-polygenic,1-trait,no pleiotropy,NaN,Est-Clines,N-cline-center-to-edge,m-constant,highly-polygenic_1-trait_no pleiotropy_nan_Est...,0-225,00500,1231094,N-cline-center-to-edge_m-constant
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,-0.350467,0.382113,oligogenic,2-trait,pleiotropy,unequal-S,SS-Mtn,N-variable,m-variable,oligogenic_2-trait_pleiotropy_unequal-S_SS-Mtn...,450-675,20000,1231768,N-variable_m-variable
96,97,-0.283708,0.382113,oligogenic,2-trait,pleiotropy,unequal-S,SS-Mtn,N-variable,m-variable,oligogenic_2-trait_pleiotropy_unequal-S_SS-Mtn...,450-675,20000,1231768,N-variable_m-variable
97,98,-0.234234,0.382113,oligogenic,2-trait,pleiotropy,unequal-S,SS-Mtn,N-variable,m-variable,oligogenic_2-trait_pleiotropy_unequal-S_SS-Mtn...,450-675,20000,1231768,N-variable_m-variable
98,99,-0.295184,0.382113,oligogenic,2-trait,pleiotropy,unequal-S,SS-Mtn,N-variable,m-variable,oligogenic_2-trait_pleiotropy_unequal-S_SS-Mtn...,450-675,20000,1231768,N-variable_m-variable


In [8]:
# exclude 1-trait 1-nuisance jobs
data = all_data[all_data['plevel'] != '1-trait']

data  # 173800 rows × 16 columns (last time)

,garden,score,final_LA,glevel,plevel,pleio,slevel,landscape,popsize,migration,simulation_level,rep,num_loci,seed,demography
0,1,-0.561309,0.501332,highly-polygenic,2-trait,no pleiotropy,equal-S,Est-Clines,N-cline-center-to-edge,m-constant,highly-polygenic_2-trait_no pleiotropy_equal-S...,0-225,00500,1231109,N-cline-center-to-edge_m-constant
1,2,-0.621278,0.501332,highly-polygenic,2-trait,no pleiotropy,equal-S,Est-Clines,N-cline-center-to-edge,m-constant,highly-polygenic_2-trait_no pleiotropy_equal-S...,0-225,00500,1231109,N-cline-center-to-edge_m-constant
2,3,-0.681061,0.501332,highly-polygenic,2-trait,no pleiotropy,equal-S,Est-Clines,N-cline-center-to-edge,m-constant,highly-polygenic_2-trait_no pleiotropy_equal-S...,0-225,00500,1231109,N-cline-center-to-edge_m-constant
3,4,-0.728768,0.501332,highly-polygenic,2-trait,no pleiotropy,equal-S,Est-Clines,N-cline-center-to-edge,m-constant,highly-polygenic_2-trait_no pleiotropy_equal-S...,0-225,00500,1231109,N-cline-center-to-edge_m-constant
4,5,-0.755431,0.501332,highly-polygenic,2-trait,no pleiotropy,equal-S,Est-Clines,N-cline-center-to-edge,m-constant,highly-polygenic_2-trait_no pleiotropy_equal-S...,0-225,00500,1231109,N-cline-center-to-edge_m-constant
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,-0.350467,0.382113,oligogenic,2-trait,pleiotropy,unequal-S,SS-Mtn,N-variable,m-variable,oligogenic_2-trait_pleiotropy_unequal-S_SS-Mtn...,450-675,20000,1231768,N-variable_m-variable
96,97,-0.283708,0.382113,oligogenic,2-trait,pleiotropy,unequal-S,SS-Mtn,N-variable,m-variable,oligogenic_2-trait_pleiotropy_unequal-S_SS-Mtn...,450-675,20000,1231768,N-variable_m-variable
97,98,-0.234234,0.382113,oligogenic,2-trait,pleiotropy,unequal-S,SS-Mtn,N-variable,m-variable,oligogenic_2-trait_pleiotropy_unequal-S_SS-Mtn...,450-675,20000,1231768,N-variable_m-variable
98,99,-0.295184,0.382113,oligogenic,2-trait,pleiotropy,unequal-S,SS-Mtn,N-variable,m-variable,oligogenic_2-trait_pleiotropy_unequal-S_SS-Mtn...,450-675,20000,1231768,N-variable_m-variable


In [9]:
data.num_loci.unique()

array(['00500', '05000', '10000', '20000'], dtype=object)

<a id='save'></a>
# save

[top](#home)

In [10]:
runtime_dir = makedir('/work/lotterhos/brandon/runtime')

runtime_dir

'/work/lotterhos/brandon/runtime'

In [11]:
f = f'{runtime_dir}/ind_performance.txt'

data.to_csv(f, sep='\t', index=False, header=True)

f

'/work/lotterhos/brandon/runtime/ind_performance.txt'

In [12]:
data.plevel.unique()

array(['2-trait'], dtype=object)

In [13]:
formatclock(dt.now() - t0)

'0-00:01:06'